In [1]:
import requests
import json
import pandas as pd

HOST = 'https://www.speedrun.com/api/v1/'

In [243]:
# get game id for majora's mask

games = requests.get('https://www.speedrun.com/api/v1/games?abbreviation=mm').text
game_dict = json.loads(games)['data'][0]

game_id = game_dict['id']
game_id

KeyError: 'data'

In [ ]:
# TEST BLOCK ADDED 1/25
game = requests.get(HOST + 'runs' + '?embed=players')
game.json()


In [51]:
# get all runs done for mm (max 200 at a time)

runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id).text
runs_dict1 = json.loads(runs)
runs_list = json.loads(runs)['data']
runs_dict1['pagination']['size'] # max elements

# new way of getting player id from run 2 (or any run) WHILE getting run
runs_dict1['data'][2]['players']['data'][0]['names']['international']

# get name of player without an account
runs_dict1['data'][12]['players']['data'][0]['name']

'Nooya'

In [69]:
# Successfull functions for first version

def make_list():
  full_list = []
  offset = 0
  runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id).text
  dict = json.loads(runs)

  while dict['pagination']['size'] == 200:
    runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id + '&offset=' + str(offset)).text
    dict = json.loads(runs)
    full_list.extend(dict['data'])
    offset += 200

  return full_list


def add_time():
  runs = make_list()
  player_times = {}

  for i in range(len(runs)):
    if runs[i]['players']['data'][0].get('names'):
      player = runs[i]['players']['data'][0]['names']['international']
    else:
      player = runs[i]['players']['data'][0]['name']


    time = runs[i]['times']['primary_t']

    if player in player_times:
      player_times[player] += time
    else:
      player_times[player] = time

  return player_times


In [87]:

def convert2(seconds):
  m, s = divmod(seconds, 60)
  h, m = divmod(m, 60)

  return str(int(h)) + ':' + str(int(m)) + ':' + str(int(s))

In [ ]:
final_data = add_time()
df = pd.DataFrame.from_dict({'Username':final_data.keys(), 'Total Time':final_data.values()})
df.sort_values(by=['Total Time'], ascending=False, inplace=True)
df['Total Time'] = df['Total Time'].apply(convert2)
df.reset_index(inplace=True, drop=True)
df

# Part 2: Generalizing to Grab Many Zelda Games

In [163]:
acronyms = ['mm', 'oot', 'tww', 'tp', 'alttp']
zelda = requests.get(HOST + 'games?name=zelda&max=100')
zelda_dict = zelda.json()['data']

# view all zelda games
zelda_games = []
for i in range (len(zelda_dict)):
    zelda_games.append(zelda_dict[i]['names']['international'])

In [219]:
GAME_SERIES = 'zelda'

def get_games(GAME_SERIES):
    id_map = {}
    request = requests.get(HOST + 'games?max=100&name=' + GAME_SERIES)
    zelda_list = request.json()['data']

    for i in range (len(zelda_list)):
        id_map[zelda_list[i]['id']] = zelda_list[i]['names']['international']

    return id_map

id_to_game = get_games(GAME_SERIES)
id_to_game

{'yd42nx6e': 'Zelda Classic',
 'o6glv55d': '3D Zelda Games',
 '3dx27j41': '2D Zelda Games',
 'w6jreedj': '3D Zelda Relay',
 'o1y0246q': "Zelda's Adventure",
 'w6jlx82d': 'Winter Solstice',
 'm1mxork6': 'BS Legend of Zelda',
 'j1n8mjx1': 'Zelda: Feng Yin Dao',
 '369p0g1l': 'The Legend of Zelda',
 '268lzk1p': 'Zelda (Game & Watch)',
 '9d3rqwed': 'Zelda 64: Dawn & Dusk',
 'nd22g3d0': 'Zelda: The Wand of Gamelon',
 '46w7n7dr': 'Minecraft: Zelda Adventure',
 'pd0wk21e': 'Zelda II: The Adventure of Link',
 'o1ynk2dq': 'The Legend of Zelda: Four Swords',
 'pdv22w96': 'The Legend of Zelda: Gerudo Exile',
 '3dx07vdy': 'The Legend of Zelda: Skyward Sword',
 'j1lore1g': 'The Legend of Zelda: Spirit Tracks',
 '268zey6p': "The Legend of Zelda: Majora's Mask",
 '9dojxo6p': 'The Legend of Zelda: Oracle of Ages',
 'w6jvl56j': 'The Legend of Zelda: The Minish Cap',
 '4d709l17': 'The Legend of Zelda: The Wind Waker',
 'k6q47pzd': 'The Legend of Zelda: Master of Time',
 'm1z98xd0': "Majora's Mask Categor

In [16]:
def new_make_list(game_id):
    full_list = []
    offset = 0
    runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id).text
    dict = json.loads(runs)

    while dict['pagination']['size'] == 200:
        runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&embed=players&game=' + game_id + '&offset=' + str(offset)).text
        dict = json.loads(runs)
        full_list.extend(dict['data'])
        offset += 200

    return full_list

# new method, make new list entry for each person, time, and game
# at the very end, sum time grouped by person and game
def add_runs(game_id):
    game_id = '268zey6p'
    runs = new_make_list(game_id)
    usernames = []
    times = []
    games = []


    for i in range(len(runs)):
        if runs[i]['players']['data'][0].get('names'):
            usernames.append(runs[i]['players']['data'][0]['names']['international'])
        else:
            usernames.append(runs[i]['players']['data'][0]['name'])

        times.append(runs[i]['times']['primary_t'])
        games.append(id_to_game[game_id])

    player_times = {
        'username' : usernames,
        'time' : times,
        'game' : games,
    }

    return player_times

In [241]:
#steps for transforming dict into final df

# df = pd.DataFrame(add_runs('stuff'), columns = ['username', 'time', 'game'])
# df = df.groupby(['username', 'game'], as_index=False).sum()
# df.sort_values(by=['time'], ascending=False, inplace=True)
# df.reset_index(drop=True, inplace=True)
# df['time'] = df['time'].apply(convert2)
# df

data = add_runs(id)
pd.DataFrame(data, columns = ['Username', 'TotalTime', 'Game'])
test = add_runs(id)
pd.DataFrame(test, columns = ['Username', 'Time', 'Game'])
#pd.DataFrame(data, columns = ['Username', 'TotalTime', 'Game'])
# df = df.groupby(['Username', 'Game'], as_index=False).sum()
# df.sort_values(by=['TotalTime'], ascending=False, inplace=True)
# df.reset_index(drop=True, inplace=True)
# df['TotalTime'] = df['TotalTime'].apply(convert2)

,Username,Time,Game


In [28]:
list(json.load(open('id-game-map.json')).keys())
run_list = new_make_list('369p0g1l')
run_list[528]['players']['data'][0] # .get('names')
#run_list
run_list[528]['id']
run_list

[{'id': '9yoegdzx',
  'weblink': 'https://www.speedrun.com/the_legend_of_zelda/run/9yoegdzx',
  'game': '369p0g1l',
  'level': None,
  'category': 'wdmw952q',
  'videos': {'links': [{'uri': 'http://www.twitch.tv/darkwing_duck_sda/c/5032646'}]},
  'comment': None,
  'status': {'status': 'verified', 'examiner': None, 'verify-date': None},
  'players': {'data': [{'rel': 'guest',
     'name': 'darkwing_duck_sda',
     'links': [{'rel': 'self',
       'uri': 'https://www.speedrun.com/api/v1/guests/darkwing_duck_sda'},
      {'rel': 'runs',
       'uri': 'https://www.speedrun.com/api/v1/runs?guest=darkwing_duck_sda'}]}]},
  'date': '2014-08-30',
  'submitted': '2014-11-26T14:15:59Z',
  'times': {'primary': 'PT30M37S',
   'primary_t': 1837,
   'realtime': 'PT30M37S',
   'realtime_t': 1837,
   'realtime_noloads': 'PT30M37S',
   'realtime_noloads_t': 1837,
   'ingame': None,
   'ingame_t': 0},
  'system': {'platform': 'jm95z9ol', 'emulated': False, 'region': 'pr184lqn'},
  'splits': None,
  'va

In [27]:
run_list[529]

{'id': 'mrq1pk4y',
 'weblink': 'https://www.speedrun.com/the_legend_of_zelda/run/mrq1pk4y',
 'game': '369p0g1l',
 'level': None,
 'category': 'w20w88dn',
 'videos': {'links': [{'uri': 'https://www.twitch.tv/videos/169376223'}]},
 'comment': 'deathless; early blue ring, early magic sword (via power bracelet)',
 'status': {'status': 'verified',
  'examiner': 'qjnzdwjm',
  'verify-date': '2017-08-30T21:44:13Z'},
 'players': {'data': []},
 'date': '2017-08-23',
 'submitted': '2017-08-30T20:55:33Z',
 'times': {'primary': 'PT52M5S',
  'primary_t': 3125,
  'realtime': 'PT52M5S',
  'realtime_t': 3125,
  'realtime_noloads': None,
  'realtime_noloads_t': 0,
  'ingame': None,
  'ingame_t': 0},
 'system': {'platform': 'jm95z9ol', 'emulated': False, 'region': 'pr184lqn'},
 'splits': None,
 'values': {'9l7p6pl1': 'zqoj63x1'},
 'links': [{'rel': 'self',
   'uri': 'https://www.speedrun.com/api/v1/runs/mrq1pk4y'},
  {'rel': 'game', 'uri': 'https://www.speedrun.com/api/v1/games/369p0g1l'},
  {'rel': 'ca